In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, auc, roc_auc_score
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import label_binarize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, RocCurveDisplay
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# 1. CNN: with_stop_word (0.6937)
DATA_PATH = "datasets/amazon_user_reviews_text_sentiment_with_sw.parquet"

# SETTINGS (instead of argparse)
TEXT_COL = "original_text"
LABEL_COL = "sentiment"

VOCAB_SIZE = 30000
MAX_LEN = 250
EMB_DIM = 128

FILTERS = 128
KERNEL_SIZES = [3, 4, 5]
DROPOUT = 0.5
DENSE_UNITS = 128

EPOCHS = 20
BATCH_SIZE = 64


# LOAD DATA
df = pd.read_parquet(DATA_PATH)

texts = df[TEXT_COL].astype(str).tolist()
y_raw = df[LABEL_COL].astype(int).to_numpy()


# Label mapping
unique = np.sort(np.unique(y_raw))
label_map = {int(v): i for i, v in enumerate(unique)}
y = np.array([label_map[int(v)] for v in y_raw])


# Split
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    texts, y, test_size=0.2, random_state=42, stratify=y
)


# TOKENIZE
tok = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tok.fit_on_texts(X_train_texts)

X_train = pad_sequences(tok.texts_to_sequences(X_train_texts), maxlen=MAX_LEN, padding="post")
X_test  = pad_sequences(tok.texts_to_sequences(X_test_texts),  maxlen=MAX_LEN, padding="post")


# BUILD CNN
inputs = tf.keras.Input(shape=(MAX_LEN,))
x = layers.Embedding(VOCAB_SIZE, EMB_DIM)(inputs)

convs = []
for k in KERNEL_SIZES:
    c = layers.Conv1D(FILTERS, k, activation="relu")(x)
    c = layers.GlobalMaxPooling1D()(c)
    convs.append(c)

x = layers.Concatenate()(convs)
x = layers.Dropout(DROPOUT)(x)
x = layers.Dense(DENSE_UNITS, activation="relu")(x)
outputs = layers.Dense(3, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


# TRAIN
history = model.fit(
    X_train, y_train,
    validation_split=0.15,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)




In [ ]:
# 2. CNN: with_stop_word + feature extended (0.695)
DATA_PATH = "datasets/amazon_user_reviews_textANDfeature_sentiment_with_sw_extended.parquet"

# SETTINGS (instead of argparse)
TEXT_COL = "original_text"
LABEL_COL = "sentiment"

VOCAB_SIZE = 30000
MAX_LEN = 250
EMB_DIM = 128

FILTERS = 128
KERNEL_SIZES = [3, 4, 5]
DROPOUT = 0.5
DENSE_UNITS = 128

EPOCHS = 20
BATCH_SIZE = 64


# LOAD DATA
df = pd.read_parquet(DATA_PATH)

texts = df[TEXT_COL].astype(str).tolist()
y_raw = df[LABEL_COL].astype(int).to_numpy()


# Label mapping
unique = np.sort(np.unique(y_raw))
label_map = {int(v): i for i, v in enumerate(unique)}
y = np.array([label_map[int(v)] for v in y_raw])


# Split
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    texts, y, test_size=0.2, random_state=42, stratify=y
)


# TOKENIZE
tok = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tok.fit_on_texts(X_train_texts)

X_train = pad_sequences(tok.texts_to_sequences(X_train_texts), maxlen=MAX_LEN, padding="post")
X_test  = pad_sequences(tok.texts_to_sequences(X_test_texts),  maxlen=MAX_LEN, padding="post")


# BUILD CNN
inputs = tf.keras.Input(shape=(MAX_LEN,))
x = layers.Embedding(VOCAB_SIZE, EMB_DIM)(inputs)

convs = []
for k in KERNEL_SIZES:
    c = layers.Conv1D(FILTERS, k, activation="relu")(x)
    c = layers.GlobalMaxPooling1D()(c)
    convs.append(c)

x = layers.Concatenate()(convs)
x = layers.Dropout(DROPOUT)(x)
x = layers.Dense(DENSE_UNITS, activation="relu")(x)
outputs = layers.Dense(3, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


# TRAIN
history = model.fit(
    X_train, y_train,
    validation_split=0.15,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)




In [ ]:
# 3. Logistic Regression: with stopword + feature (0.7016)

# load extended data for text sentiment
df = pd.read_parquet('datasets/amazon_user_reviews_textANDfeature_sentiment_with_sw.parquet')


# Drop rows where 'original_text' is NaN or empty string
df.dropna(subset=['original_text'], inplace=True)
df = df[df['original_text'].str.strip() != '']

# Split data into features (X) and target (y)
X = df['original_text']
y = df['sentiment']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training data size: {X_train.shape[0]} samples")
print(f"Testing data size: {X_test.shape[0]} samples")

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

# Fit and transform the training data, transform the test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train Logistic Regression model
lr_model = LogisticRegression(max_iter=1000, solver='liblinear', random_state=42)
lr_model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(X_test_tfidf)
